In [ ]:
import camelot
import pandas as pd
import requests
import zipfile
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from datetime import date

In [ ]:
#Define date of today used format
d=date.today()
date_str=d.strftime("%d%m%Y")

#Define a manual date (e.g. if running on a date where no new data is uploaded)
#date_str="24122020"

In [ ]:
# Find link to latest file from the web page
# assumes stabel initial part of filename and webpage location
#req = Request("https://www.ssi.dk/sygdomme-beredskab-og-forskning/sygdomsovervaagning/c/covid19-overvaagning")
req = Request("https://covid19.ssi.dk/overvagningsdata/private-tests")
html_page = urlopen(req)
soup = BeautifulSoup(html_page, "html")
                     
for link in soup.findAll('a'):
    ref=link.get('href')
    if isinstance(ref, str) and "opgoerelse-pcr-og-antigentest-05012021" in ref.lower() and date_str in ref.lower(): # NB compare in lower case
        print("url for zip file: " + link.get('href'))
        url=link.get('href')
        #date_str = url[50:58]

        

In [ ]:
# Direct download data and unpacking
r = requests.get(url, allow_redirects=True)
open('data_privat.pdf', 'wb').write(r.content)
PU_file='data_privat.pdf'

In [ ]:
# Manually define PDF file to extract tables from in case above download breaks
#PU_file = "Opgoerelse-pcr-og-antigentest-05012021-fsda.pdf"


In [ ]:
PU_tables = camelot.read_pdf(PU_file, pages = '2', flavor='stream')

In [ ]:
PU_data=PU_tables[0].df

In [ ]:
PU_data

In [ ]:
#remove wrong rows
PU_data=PU_data[5:-1]
PU_data

In [ ]:
PU_data.columns = ['Date', 'Antal testudbydere', 'Antal PCR tests' , 'Antal positiv PCR tests','Positiv procent PCR' ,'Antal antigen tests','Antal positiv antigen tests','Positiv procent antigen']


In [ ]:
PU_data['Date']=pd.to_datetime(PU_data['Date'], format='%d.%m.%Y')
PU_data['Antal PCR tests'] = PU_data['Antal PCR tests'].str.replace('.', '').astype(float)
PU_data['Antal positiv PCR tests'] = PU_data['Antal positiv PCR tests'].str.replace('.', '').astype(float)
PU_data['Positiv procent PCR'] = PU_data['Positiv procent PCR'].str.replace(',', '.').str.replace('%', '').astype(float)
PU_data['Antal antigen tests'] = PU_data['Antal antigen tests'].str.replace('.', '').astype(float)
PU_data['Antal positiv antigen tests'] = PU_data['Antal positiv antigen tests'].str.replace('.', '').astype(float)
PU_data['Positiv procent antigen'] = PU_data['Positiv procent antigen'].str.replace(',', '.').str.replace('%', '').astype(float)
PU_data=PU_data.set_index(['Date'])

In [ ]:
PU_data

In [ ]:
PU_data.to_pickle('data_private.dat')

In [ ]:
PU_data.plot(y='Positiv procent antigen',style='.',color='green',label='From "Antigen Tests" (Quick tests)');